# Bosch Small Traffic Lights Dataset

## Convert labels

In [12]:
train_labels_dir = "traffic_light_data/labels/train"
test_labels_dir = "traffic_light_data/labels/test"

In [ ]:
!mkdir {train_labels_dir}
!mkdir {test_labels_dir}

In [22]:
!python convert_labels.py --input traffic_light_data/train.yaml --output_dir {train_labels_dir} --filter off
!python convert_labels.py --input traffic_light_data/test.yaml --output_dir {test_labels_dir} --filter off

Bounding boxes: 10756
Bounding boxes after filtering: 10030
Bounding boxes: 13486
Bounding boxes after filtering: 13044


## Split data

Open the data folder.

In [25]:
%cd traffic_light_data

C:\Users\diogo\Desktop\DIP-Project\traffic_light_data


Create directories for the validation set.

In [26]:
!mkdir "images/val"
!mkdir "labels/val"

Go back to the parent directory.

In [27]:
%cd ..

C:\Users\diogo\Desktop\DIP-Project


Split the test set into validation and test sets, with a 50/50 split.

In [32]:
!python split_data.py --data "traffic_light_data" --ratio 0.5

Images in traffic_light_data\images: 0
Images in traffic_light_data\images\val: 0
Labels in traffic_light_data\labels: 0
Labels in traffic_light_data\labels\test: 8334
Labels in traffic_light_data\labels\train: 5093
Labels in traffic_light_data\labels\val: 0


Create test video from test images, at a framerate of 15 fps (as per the original video).

In [ ]:
!python generate_video.py --input "traffic_light_data/images/test" --fps 15

Namespace(fps=15, size=[1280, 720], fourcc='mp4v', input='../traffic_light_data/images/test', output='../traffic_light_data/images/test/test.mp4')


# YOLOv5

## Clone the repository

In [3]:
!git clone "https://github.com/ultralytics/yolov5"

Cloning into 'yolov5'...


In [9]:
%cd yolov5

C:\Users\ImageLab\Desktop\DIP-Project\yolov5


In [5]:
%pip install -r requirements.txt

     ------------------------------------- 184.0/184.0 kB 11.6 MB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Transfer Learning

Finetune YOLOv5-S with 4 classes: Green, Yellow, Red, and off.

In [ ]:
!python train.py --batch 32 --epochs 100 --data custom_with_off.yaml --weights yolov5s.pt --cache ram

Finetune YOLOv5-S with 3 classes: Green, Yellow, and Red.

In [ ]:
!python train.py --batch 32 --epochs 100 --data custom.yaml --weights yolov5s.pt --cache ram

## Evaluation

Evaluate the fine-tuned YOLOv5-S with 4 classes on the test set.

In [26]:
!python val.py --weights "runs/train/exp/weights/best.pt" --data custom_with_off.yaml --name exp

val: data=C:\Users\ImageLab\Desktop\Diogo\yolov5\data\custom_with_off.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v7.0-71-gc442a2e Python-3.9.16 torch-1.13.1 CUDA:0 (Quadro RTX 8000, 49152MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs

val: Scanning C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\labels\val...:   0%|          | 0/4167 [00:00<?, ?it/s]
val: Scanning C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\labels\val... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/4167 [00:03<3:36:17,  3.12s/it]
val: Scanning C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\labels\val... 50 images, 33 backgrounds, 0 corrupt:   1%|1      

Evaluate the fine-tuned YOLOv5-S with 3 classes on the test set.

In [6]:
!python val.py --weights "runs/train/exp2/weights/best.pt" --data custom.yaml --name exp2

val: data=C:\Users\ImageLab\Desktop\Diogo\yolov5\data\custom.yaml, weights=['runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v7.0-71-gc442a2e Python-3.9.16 torch-1.13.1 CUDA:0 (Quadro RTX 8000, 49152MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs

val: Scanning C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\labels\val...:   0%|          | 0/4167 [00:00<?, ?it/s]
val: Scanning C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\labels\val... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/4167 [00:03<3:31:11,  3.04s/it]
val: Scanning C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\labels\val... 144 images, 127 backgrounds, 0 corrupt:   3%|3         | 1

## Inference

Load the fine-tuned YOLOv5-S weights and infer on the test video.

In [ ]:
test_video = "../traffic_light_data/videos/test.mp4"

In [27]:
!python detect.py --weights "runs/train/exp/weights/best.pt" --source {test_video} --name exp

detect: weights=['runs/train/exp/weights/best.pt'], source=../traffic_light_data/videos/test.mp4, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-71-gc442a2e Python-3.9.16 torch-1.13.1 CUDA:0 (Quadro RTX 8000, 49152MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/4167) C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\videos\test.mp4: 384x640 (no detections), 15.8ms
video 1/1 (2/4167) C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\videos\test.mp4: 384x640 (no detections), 6.7ms
video 1/1 (3/4167) C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\videos\test.mp4: 384x6

Load the fine-tuned YOLOv5-S weights trained on 3 classes and infer on the test video.

In [7]:
!python detect.py --weights "runs/train/exp2/weights/best.pt" --source {test_video} --name exp2

detect: weights=['runs/train/exp2/weights/best.pt'], source=../traffic_light_data/videos/test.mp4, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp2, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-71-gc442a2e Python-3.9.16 torch-1.13.1 CUDA:0 (Quadro RTX 8000, 49152MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/4167) C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\videos\test.mp4: 384x640 (no detections), 13.7ms
video 1/1 (2/4167) C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\videos\test.mp4: 384x640 (no detections), 12.6ms
video 1/1 (3/4167) C:\Users\ImageLab\Desktop\Diogo\traffic_light_data\videos\test.mp4: 38

Display inferred video.

In [ ]:
%cd ..

In [10]:
!python display_video.py --input "yolov5/runs/detect/exp/test.mp4"

Namespace(input='runs/detect/exp/test.mp4', fps=15)


In [11]:
!python display_video.py --input "yolov5/runs/detect/exp2/test.mp4"

Namespace(input='runs/detect/exp2/test.mp4', fps=15)


## Upload data

In [ ]:
!gsutil -m cp -r "runs/*" "gs://traffic_light_data/runs/yolov5"

# YOLOv6

## Clone the repository

In [14]:
%cd ..

C:\Users\ImageLab\Desktop\DIP-Project


In [23]:
!git clone "https://github.com/meituan/YOLOv6" yolov6

Cloning into 'YOLOv6'...


In [33]:
%cd yolov6

C:\Users\diogo\Desktop\DIP-Project\yolov6


In [ ]:
%pip install -r requirements.txt

## Training

Finetune YOLOv6-S with 4 classes: Green, Yellow, Red, and off.

In [12]:
!python "tools/train.py" --batch 32 \
    --conf "configs/yolov6s_finetune.py" \
    --epochs 100 \
    --data "data/dataset.yaml" \
    --device 0

^C


Finetune YOLOv6-S with 3 classes: Green, Yellow, and Red.

In [12]:
!python "tools/train.py" --batch 32 --conf "configs/yolov6s_finetune.py" --epochs 100 --data "data/dataset_without_off.yaml" --device 0

^C


## Evaluation

In [14]:
!python "tools/eval.py" --data "data/dataset.yaml" --weights "runs/train/exp/weights/best_ckpt.pt" --device 0 --name exp

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.40s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.34s).
Accumulating evaluation results...
DONE (t=1.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.176
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.455
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.166
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.392
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.156
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.229
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

Namespace(data='data/dataset.yaml', weights='runs/train/exp/weights/best_ckpt.pt', batch_size=32, img_size=640, conf_thres=0.03, iou_thres=0.65, task='val', device='0', half=False, save_dir='runs/val/', name='exp', test_load_size=640, letterbox_return_int=False, scale_exact=False, force_no_pad=False, not_infer_on_rect=False, reproduce_640_eval=False, eval_config_file='./configs/experiment/eval_640_repro.py', do_coco_metric=True, do_pr_metric=False, plot_curve=True, plot_confusion_matrix=False, verbose=False, config_file='')
Loading checkpoint from runs/train/exp/weights/best_ckpt.pt

Fusing model...
C:\Users\ImageLab\anaconda3\envs\myenv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Switch model to deploy modality.
M

In [12]:
!python "tools/eval.py" --data "data/dataset_without_off.yaml" --weights "runs/train/exp1/weights/best_ckpt.pt" --device 0 --name exp1

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.44s).
Accumulating evaluation results...
DONE (t=0.60s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.278
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.678
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.270
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.621
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.233
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.339
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

Namespace(data='data/dataset_without_off.yaml', weights='runs/train/exp1/weights/best_ckpt.pt', batch_size=32, img_size=640, conf_thres=0.03, iou_thres=0.65, task='val', device='0', half=False, save_dir='runs/val/', name='exp1', test_load_size=640, letterbox_return_int=False, scale_exact=False, force_no_pad=False, not_infer_on_rect=False, reproduce_640_eval=False, eval_config_file='./configs/experiment/eval_640_repro.py', do_coco_metric=True, do_pr_metric=False, plot_curve=True, plot_confusion_matrix=False, verbose=False, config_file='')
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
C:\Users\ImageLab\anaconda3\envs\myenv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Switch model to dep

## Inference

Infer on the test video and save the output video.

In [ ]:
!python "tools/infer.py" \
    --yaml "data/dataset.yaml" \
    --weights "runs/train/exp/weights/best_ckpt.pt" \
    --source "../traffic_light_data/videos/test.mp4" \
    --name "exp" \
    --device 0

Infer on the test video with the YOLOv6-S weights trained on the 3 classes.

In [3]:
!python "tools/infer.py" --yaml "data/dataset_without_off.yaml" --weights "runs/train/exp1/weights/best_ckpt.pt" --source "../traffic_light_data/videos/test.mp4" --name "exp1" --device 0

Namespace(weights='runs/train/exp1/weights/best_ckpt.pt', source='../traffic_light_data/videos/test.mp4', webcam=False, webcam_addr='0', yaml='data/dataset_without_off.yaml', img_size=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device='0', save_txt=False, not_save_img=False, save_dir=None, view_img=False, classes=None, agnostic_nms=False, project='runs/inference', name='exp1', hide_labels=False, hide_conf=False, half=False)
Save directory already existed
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
Switch model to deploy modality.
c:\Users\ImageLab\anaconda3\envs\myenv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

  0%|          | 0/1 [00:00<?, ?it/s]
4it [00:00, 37.78i

Display inferred videos side by side.

In [34]:
%cd ..

C:\Users\diogo\Desktop\DIP-Project


In [16]:
!python display_video.py --input "yolov6/runs/inference/exp/test.mp4"

Namespace(input='runs/inference/exp/test.mp4', fps=15)


In [17]:
!python display_video.py --input "yolov6/runs/inference/exp1/test.mp4"

Namespace(input='runs/inference/exp1/test.mp4', fps=15)


## Upload data

In [9]:
!gsutil -m cp -r "runs/*" "gs://traffic_light_data/runs/yolov6"

Copying file://YOLOv6\runs\inference\exp\test.mp4 [Content-Type=video/mp4]...
/ [0 files][    0.0 B/151.8 MiB]                                                
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
- [0 files][ 68.4 MiB/151.8 MiB]                                                
\
\ [1 files][151.8 MiB/151.8 MiB]                                                

# YOLOv7

## Clone the repository

In [23]:
%cd ..

C:\Users\ImageLab\Desktop\DIP-Project


In [79]:
!git clone "https://github.com/WongKinYiu/yolov7.git"

Cloning into 'yolov7'...


In [24]:
%cd yolov7

C:\Users\ImageLab\Desktop\DIP-Project\yolov7


In [82]:
%pip install -r requirements.txt

     --------------------------------------- 10.9/10.9 MB 93.0 MB/s eta 0:00:00
     ---------------------------------------- 293.3/293.3 kB ? eta 0:00:00


## Transfer Learning

Finetune YOLOv7 with 4 classes: Green, Yellow, Red, and off.

In [ ]:
!python train.py --device 0 --epochs 100 --batch-size 32 --data "data/custom.yaml" --cfg "cfg/training/yolov7.yaml" --weights 'yolov7_training.pt' --hyp "data/hyp.scratch.custom.yaml"

Finetune YOLOv7 with 3 classes: Green, Yellow, and Red.

In [ ]:
!python train.py --device 0 --epochs 100 --batch-size 32 --data "data/without_off.yaml" --cfg "cfg/training/yolov7.yaml" --weights 'yolov7_training.pt' --hyp "data/hyp.scratch.custom.yaml"

## Evaluation

In [20]:
!python test.py --data "data/custom.yaml" --weights "runs/train/exp/weights/best.pt" --name exp

Namespace(weights=['runs/train/exp/weights/best.pt'], data='data/custom.yaml', batch_size=32, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all        4167        5586       0.837       0.451       0.454       0.138
               Green        4167        3779       0.889       0.871       0.891        0.31
                 Red        4167        1500       0.621       0.481       0.472       0.105
                 off        4167         307           1           0    3.85e-06    7.12e-07
Speed: 2.9/1.1/4.0 ms inference/NMS/total per 640x640 image at batch-s

YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1 CUDA:0 (Quadro RTX 8000, 49151.5625MB)

C:\Users\ImageLab\anaconda3\envs\myenv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36497954 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/4167 [00:00<?, ?it/s]
val: Scanning '..\traffic_light_data\labels\val' images and labels... 54 found, 0 missing, 37 empty, 0 corrupted:   1%|1         | 54/4167 [00:00<00:08, 502.58it/s]
val: Scanning '..\traffic_light_data\labels\val' images and labels... 110 found, 0 missing, 93 empty, 0 corrupted:   3%|2         | 110/4167 [00:00<00:08, 502.77it/s]
val: Scanning '..\traffic_light_data\labels\val' images and labels... 165 found, 0 missing, 

In [21]:
!python test.py --data "data/without_off.yaml" --weights "runs/train/exp2/weights/best.pt" --name exp2

Namespace(weights=['runs/train/exp2/weights/best.pt'], data='data/without_off.yaml', batch_size=32, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp2', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all        4167        5279       0.617       0.645       0.562       0.185
               Green        4167        3779       0.811       0.866       0.864       0.309
                 Red        4167        1500       0.422       0.423       0.259      0.0598
Speed: 2.8/1.1/3.9 ms inference/NMS/total per 640x640 image at batch-size 32
Results saved to runs\test\exp2


YOLOR  v0.1-121-g2fdc7f1 torch 1.13.1 CUDA:0 (Quadro RTX 8000, 49151.5625MB)

C:\Users\ImageLab\anaconda3\envs\myenv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36492560 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/4167 [00:00<?, ?it/s]
val: Scanning '..\traffic_light_data\labels\val' images and labels... 49 found, 0 missing, 32 empty, 0 corrupted:   1%|1         | 49/4167 [00:00<00:09, 441.66it/s]
val: Scanning '..\traffic_light_data\labels\val' images and labels... 95 found, 0 missing, 78 empty, 0 corrupted:   2%|2         | 95/4167 [00:00<00:09, 450.98it/s]
val: Scanning '..\traffic_light_data\labels\val' images and labels... 148 found, 0 missing, 13

## Inference

Load the fine-tuned YOLOv7 weights and infer on the test video.

In [ ]:
!python detect.py --device 0 --conf 0.25 --source {test_video} --weights "runs/train/exp/weights/best.pt"

Load the fine-tuned YOLOv7 weights trained on 3 classes and infer on the test video.

In [ ]:
!python detect.py --device 0 --conf 0.25 --source {test_video} --weights "runs/train/exp2/weights/best.pt"

Display inferred video.

In [37]:
%cd ..

C:\Users\diogo\Desktop\DIP-Project


In [42]:
!python display_video.py --input "yolov7/runs/detect/exp/test.mp4"

Namespace(input='traffic_light_data/videos/yolov5_no_off.mp4', fps=15)


[ WARN:0@0.361] global C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\videoio\src\cap_gstreamer.cpp (2386) cv::handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@0.361] global C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\videoio\src\cap_gstreamer.cpp (2402) cv::handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@0.361] global C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\videoio\src\cap_gstreamer.cpp (1356) cv::GStreamerCapture::open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@0.361] global C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\videoio\src\cap_gstreamer.cpp (862) cv::GStreamerCapture::isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [26]:
!python display_video.py --input "yolov7/runs/detect/exp2/test.mp4"

Namespace(input='runs/detect/exp2/test.mp4', fps=15)


## Upload data

In [2]:
!gsutil -m cp -r "runs/*" "gs://traffic_light_data/runs/yolov7"

Copying file://yolov7\runs\detect\exp\test.mp4 [Content-Type=video/mp4]...
/ [0 files][    0.0 B/145.0 MiB]                                                
-
- [0 files][ 58.8 MiB/145.0 MiB]                                                
\
\ [1 files][145.0 MiB/145.0 MiB]                                                
|

Operation completed over 1 objects/145.0 MiB.                                    
